# Importing the libraries

In [13]:
import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import MNIST
from torch.optim import AdamW
import torchvision.transforms as tf
import numpy as np

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
print(torch.__version__)

1.13.1+cpu


# Define the data transforms

In [12]:
train_tf = tf.Compose([tf.Resize((224, 224)),
                       tf.RandomHorizontalFlip(p=0.5),
                       tf.ToTensor(),
                       tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

val_tf = tf.Compose([tf.Resize((224, 224)),
                    tf.ToTensor(),
                    tf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

# Get the MNIST dataset

In [ ]:
train_dataset = MNIST(root='./data', train=True, download=True, transform=train_tf)
test_dataset = MNIST(root='./data', train=False, download=True, transform=val_tf)

Failed to download (trying next):
HTTP Error 404: Not Found

